In [ ]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 10
num_captions = 3  # You can change this number to generate more captions per image
gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "num_return_sequences": num_captions}

def predict_captions(image_path):
    try:
        image = Image.open(image_path)
        if image.mode != "RGB":
            image = image.convert(mode="RGB")
        pixel_values = feature_extractor(images=[image], return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)
        output_ids = model.generate(pixel_values, **gen_kwargs)
        
        # Split the generated captions into a list
        all_preds = []
        for seq_ids in output_ids:
            preds = tokenizer.decode(seq_ids, skip_special_tokens=True)
            preds = [pred.strip() for pred in preds.split('\n')]
            all_preds.extend(preds)
        
        return all_preds
    except Exception as e:
        print("Error: ", str(e))

# Example usage:
image_path = "path_to_your_image.jpg"
predictions = predict_captions(image_path)
for caption in predictions:
    print(caption) 
